In [ ]:
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

In [ ]:
# %cd drive/MyDrive/documents/images/momseguros_tucarro/

In [ ]:
# %ls

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import json

In [ ]:
batch_size = 32
img_size = (256, 256)

model_name = ['oi_', '_w_v1']

In [ ]:
images_train_directory = '../images/oi/train'
images_val_directory = '../images/oi/validation'
# images_test_directory = './images/oi/test'

In [ ]:
class_names = ['inside', 'outside']

train_ds = tf.keras.utils.image_dataset_from_directory(
  images_train_directory,
  class_names=class_names,
  seed=123,
  image_size=img_size,
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  images_val_directory,
  class_names=class_names,
  shuffle=False,
  image_size=img_size,
  batch_size=batch_size
)

# test_ds = tf.keras.utils.image_dataset_from_directory(
#   images_test_directory,
#   class_names=class_names,
#   shuffle=False,
#   image_size=img_size,
#   batch_size=batch_size
# )         

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(10):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
])

for i in train_ds.take(1):
    image = i[0][0]

tf.cast(tf.expand_dims(image, 0), tf.float32)
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmentation(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(tf.keras.utils.array_to_img(augmented_image))
  plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

# train_ds = train_ds.cache()
# val_ds = val_ds.cache()
# test_ds = test_ds.cache()

In [ ]:
# callbacks
class AccReached(tf.keras.callbacks.Callback):
  def __init__(self):
    pass
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.99):
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True, mode='min', verbose=1)

check_point_callback = tf.keras.callbacks.ModelCheckpoint(
  filepath=f'models/{model_name[0]}model{model_name[1]}.h5',
  monitor='val_loss',
  mode='min',
  save_best_only=True,
  save_weights_only=True,
)

In [ ]:
# learning rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
  initial_learning_rate=0.001,
  decay_steps=2000,
  decay_rate=0.9
  )

In [ ]:
REGULARIZATION_LAMBDA = 0.000015

model = tf.keras.models.Sequential([
    data_augmentation,
    tf.keras.layers.Rescaling(1./255, input_shape=(256, 256, 3)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3042, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2028, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1014, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(
  loss='binary_crossentropy',
  optimizer=tf.optimizers.SGD(learning_rate=lr_schedule),
  metrics=['accuracy'],
)

In [ ]:
history = model.fit(
  train_ds,
  steps_per_epoch=32,
  epochs=1000,
  verbose=1,
  callbacks=[AccReached(), check_point_callback],
  validation_data=val_ds,
)

In [ ]:
with open(f'models/{model_name[0]}history{model_name[1]}.json', 'w') as json_file:
    json.dump(history.history, json_file)